In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [3]:
year_months = list(uselog_months['연월'].unique())
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"] == year_months[i]]
    tmp.rename(columns = {"count":"count_0"}, inplace = True)
    tmp_before = uselog_months.loc[uselog_months["연월"] == year_months[i-1]]
    del tmp_before["연월"]
    tmp_before.rename(columns = {"count":"count_1"}, inplace = True)
    tmp = pd.merge(tmp, tmp_before, on = 'customer_id', how = 'left')
    uselog = pd.concat([uselog, tmp], ignore_index = True)
uselog.head()    

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [4]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"] == 1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months = 1)

exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on = ['customer_id','연월'], how = "left")
print(len(exit_uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
exit_uselog = exit_uselog.dropna(subset = ["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [7]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on = ['customer_id'], how = "left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset = ["name"])
print(len(conti_uselog))

33851
27422


In [8]:
conti_uselog

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201805,AS002855,5,4.0,XXXX,C03,F,2016-11-01,NaN,CA1,...,야간,6000.0,일반,4.500000,5.0,7.0,2.0,1.0,2019-04-30,29.0
1,201805,AS009373,4,3.0,XX,C01,F,2015-11-01,NaN,CA1,...,종일,10500.0,일반,5.083333,5.0,7.0,3.0,1.0,2019-04-30,41.0
2,201805,AS015233,7,NaN,XXXXX,C01,M,2018-05-13,NaN,CA2,...,종일,10500.0,입회비반액할인,7.545455,7.0,11.0,4.0,1.0,2019-04-30,11.0
3,201805,AS015315,3,6.0,XXXXX,C01,M,2015-07-01,NaN,CA1,...,종일,10500.0,일반,4.833333,5.0,7.0,3.0,1.0,2019-04-30,45.0
4,201805,AS015739,5,7.0,XXXXX,C03,M,2017-06-01,NaN,CA1,...,야간,6000.0,일반,5.583333,5.5,8.0,4.0,1.0,2019-04-30,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33846,201903,TS995853,8,11.0,XXXX,C01,M,2019-02-08,NaN,CA1,...,종일,10500.0,일반,9.500000,9.5,11.0,8.0,1.0,2019-04-30,2.0
33847,201903,TS998593,8,7.0,XXXXX,C03,M,2018-09-01,NaN,CA1,...,야간,6000.0,일반,8.142857,8.0,9.0,7.0,1.0,2019-04-30,7.0
33848,201903,TS999079,3,2.0,XXX,C03,M,2016-06-01,NaN,CA1,...,야간,6000.0,일반,4.916667,5.5,9.0,2.0,1.0,2019-04-30,34.0
33849,201903,TS999231,6,6.0,XXXX,C01,M,2017-03-01,NaN,CA1,...,종일,10500.0,일반,4.666667,5.0,8.0,1.0,1.0,2019-04-30,25.0


In [9]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset = "customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201810,IK761183,8,8.0,XXXX,C02,F,2018-06-12,NaN,CA2,...,주간,7500.0,입회비반액할인,8.100000,8.0,10.0,6.0,1.0,2019-04-30,10.0
1,201807,AS124354,7,5.0,XXXX,C03,M,2015-09-01,NaN,CA1,...,야간,6000.0,일반,4.583333,4.0,7.0,3.0,1.0,2019-04-30,43.0
2,201812,TS021036,8,5.0,XXXXX,C01,M,2016-09-01,NaN,CA1,...,종일,10500.0,일반,5.166667,5.0,8.0,1.0,1.0,2019-04-30,31.0
3,201809,OA816679,5,7.0,XXXX,C03,M,2016-06-01,NaN,CA1,...,야간,6000.0,일반,4.750000,5.0,7.0,3.0,1.0,2019-04-30,34.0
4,201809,HD769845,4,5.0,XXXX,C03,M,2016-02-01,NaN,CA1,...,야간,6000.0,일반,4.666667,5.0,7.0,2.0,1.0,2019-04-30,38.0


In [12]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index = True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201810,IK761183,8,8.0,XXXX,C02,F,2018-06-12,NaN,CA2,...,7500.0,입회비반액할인,8.100000,8.0,10.0,6.0,1.0,2019-04-30,10.0,NaN
1,201807,AS124354,7,5.0,XXXX,C03,M,2015-09-01,NaN,CA1,...,6000.0,일반,4.583333,4.0,7.0,3.0,1.0,2019-04-30,43.0,NaN
2,201812,TS021036,8,5.0,XXXXX,C01,M,2016-09-01,NaN,CA1,...,10500.0,일반,5.166667,5.0,8.0,1.0,1.0,2019-04-30,31.0,NaN
3,201809,OA816679,5,7.0,XXXX,C03,M,2016-06-01,NaN,CA1,...,6000.0,일반,4.750000,5.0,7.0,3.0,1.0,2019-04-30,34.0,NaN
4,201809,HD769845,4,5.0,XXXX,C03,M,2016-02-01,NaN,CA1,...,6000.0,일반,4.666667,5.0,7.0,2.0,1.0,2019-04-30,38.0,NaN


In [13]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"],format = "%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)

predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201810,IK761183,8,8.0,XXXX,C02,F,2018-06-12,NaN,CA2,...,8.100000,8.0,10.0,6.0,1.0,2019-04-30,10.0,NaN,3,2018-10-01
1,201807,AS124354,7,5.0,XXXX,C03,M,2015-09-01,NaN,CA1,...,4.583333,4.0,7.0,3.0,1.0,2019-04-30,43.0,NaN,34,2018-07-01
2,201812,TS021036,8,5.0,XXXXX,C01,M,2016-09-01,NaN,CA1,...,5.166667,5.0,8.0,1.0,1.0,2019-04-30,31.0,NaN,27,2018-12-01
3,201809,OA816679,5,7.0,XXXX,C03,M,2016-06-01,NaN,CA1,...,4.750000,5.0,7.0,3.0,1.0,2019-04-30,34.0,NaN,27,2018-09-01
4,201809,HD769845,4,5.0,XXXX,C03,M,2016-02-01,NaN,CA1,...,4.666667,5.0,7.0,2.0,1.0,2019-04-30,38.0,NaN,31,2018-09-01


In [15]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               263
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [16]:
predict_data = predict_data.dropna(subset = ['count_1'])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2631
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2631
period                  0
now_date                0
dtype: int64

In [18]:
target_col = ["campaign_name","class_name","gender","count_1","routine_flg","period","is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,입회비반액할인,주간,F,8.0,1.0,3,0.0
1,일반,야간,M,5.0,1.0,34,0.0
2,일반,종일,M,5.0,1.0,27,0.0
3,일반,야간,M,7.0,1.0,27,0.0
4,일반,야간,M,5.0,1.0,31,0.0


In [19]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,8.0,1.0,3,0.0,0,0,1,0,0,1,1,0
1,5.0,1.0,34,0.0,1,0,0,1,0,0,0,1
2,5.0,1.0,27,0.0,1,0,0,0,1,0,0,1
3,7.0,1.0,27,0.0,1,0,0,1,0,0,0,1
4,5.0,1.0,31,0.0,1,0,0,1,0,0,0,1


In [20]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,8.0,1.0,3,0.0,0,1,0,1,1
1,5.0,1.0,34,0.0,0,0,0,0,0
2,5.0,1.0,27,0.0,0,0,1,0,0
3,7.0,1.0,27,0.0,0,0,0,0,0
4,5.0,1.0,31,0.0,0,0,0,0,0
